In [17]:
import numpy as np
from collections import Counter

In [112]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf(self):
        return self.value is not None
        
        
class DecisionTree:
    def __init__(self, min_samples_split=3, max_depth=101, num_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.num_features = num_features
        self.root = None
        
    def fit(self, X, y):
        self.num_features = X.shape[1] if not self.num_features else min(X.shape[1], self.num_features)
        self.root = self._expand_tree(X, y)
    
    def _expand_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        num_labels = len(np.unique(y))
        if depth >= self.max_depth or num_labels == 1 or num_samples<self.min_samples_split:
            leaf_value = self.get_most_common_label(y)
            return Node(value=leaf_value)
        features_idx = np.random.choice(num_features, self.num_features, replace=False)
        best_feature, best_thresh = self.best_split(X, y, features_idx)
        left_idx, right_idx = self._split(X[:, best_feature], best_thresh)
        left = self._expand_tree(X[left_idx, :], y[left_idx], depth+1)
        right = self._expand_tree(X[right_idx, :], y[right_idx], depth+1)
        return Node(best_feature, best_thresh, left, right)
    
    def best_split(self, X, y, features_idx):
        best_gain = -1
        split_idx, split_thresh = None, None
        for f in features_idx:
            X_col = X[:, f]
            threshs = np.unique(X_col)
            for th in threshs:
                gain = self._generate_information_gain(y, X_col, th)
                if gain > best_gain:
                    best_gain = gain 
                    split_idx = f
                    split_thresh = th
        return split_idx, split_thresh
    
    def _generate_information_gain(self, y, X_column, threshold):
        parent_entropy = self._entropy(y)
        left_idxs, right_idxs = self._split(X_column, threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r
        information_gain = parent_entropy - child_entropy
        return information_gain     
        
    def _split(self, X_col, threshold):
        left_idx = np.argwhere(X_col<=threshold).flatten()
        right_idx = np.argwhere(X_col>threshold).flatten()
        return left_idx, right_idx
        
    def _entropy(self, y):
        count_vals = np.bincount(y)
        ps = count_vals / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])
        
    def get_most_common_label(self, y):
        count = Counter(y)
        return count.most_common(1)[0][0]
        
    def _traverse_tree(self, x, node):
        if node.is_leaf():
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
        
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])
        

In [113]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [114]:
data = datasets.load_breast_cancer()
X, y = data.data, data.target

In [115]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [116]:
clf = DecisionTree(max_depth=10)

In [117]:
clf.fit(X_train, y_train)

In [118]:
predictions = clf.predict(X_test)

In [119]:
def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred)/len(y_test)

In [120]:
accuracy(y_test, predictions)

0.9385964912280702